In [2]:
# do pip installs to get libraries: wordfreq, googletrans, editdistance, unidecode

from wordfreq import word_frequency
from googletrans import Translator
import editdistance
from unidecode import unidecode

import time

translator = Translator()

This script will process the data files to add information about translations and frequencies.

In [51]:
lang='en_es'
fn = 'data/data_{0}/{0}.slam.20171218.train.new'.format(lang)
(src, dest)=lang.split('_')
print("Translating %s to %s" %(src,dest))


Translating en to es


In [52]:
fn

'data/data_en_es/en_es.slam.20171218.train.new'

In [5]:
#Todd's code
# trans_cache = {}

# newlines=[]
# with open(fn, 'r') as f:
#     for line in f:
#         if len(line)>1:
#             #print("***"+line)
#             if line[0]!='#':
#                 # process and translate
#                 pieces = line.split()
#                 idx = pieces[0]
#                 word = pieces[1]
#                 root_token = pieces[2]
#                 part_of_speech = pieces[3]
#                 morph_feats = pieces[4]
#                 dependency_label = pieces[5]
#                 dependency_edge_head = pieces[6]
                
#                 clean_word = word.lower()
#                 if '\'' not in clean_word:
#                     if clean_word not in trans_cache:
#                         translation_obj = translator.translate(clean_word,src=src,dest=dest)
#                         translation = translation_obj.text
#                         print("%s to %s" % (clean_word, translation))  # need to do some clean up of the translation
#                         trans_cache[clean_word]=translation
#                     else:
#                         translation = trans_cache[clean_word]
                
#                 trans_freq = word_frequency(translation, dest)
#                 src_frec = word_frequency(word, src)
                
                
#                 if len(pieces) == 8:
#                     label = pieces[7]
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head, label]
#                 else:
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head]
#                 newlines.append("\t".join(repl)+'\n')
#             else:
#                 newlines.append(line)
#         else:
#             newlines.append(line)

# print(len(newlines))
# with open('test.txt','w') as fp:
#     fp.writelines(newlines)

In [7]:
import pandas as pd
import numpy as np

In [29]:
aoa_xl = pd.ExcelFile("data/13428_2013_348_MOESM1_ESM.xlsx")
aoa_df = aoa_xl.parse('Sheet1')

In [47]:
#Pam's code
# FOR EVERY ROOT WORD (e.g, "be" instead of "is")
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is

#output filename
fnout = "{}_{}_rootwordfeats.txt".format(src,dest)
print("Will save translations and features to: ",fnout)


trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1 and line[0]!='#':
            # process and translate
            pieces = line.split()
            idx = pieces[0]
            word = pieces[1]
            root_token = pieces[2]
            part_of_speech = pieces[3]
            morph_feats = pieces[4]
            dependency_label = pieces[5]
            dependency_edge_head = pieces[6]

            clean_word = root_token.lower() # Only using root token here rather than word
            if '\'' not in clean_word and clean_word not in trans_cache:
                # Wait so as not to annoy the API (otherwise will give JSON error)
                time.sleep(0.2)

                # Translate so we know the word in the language the user already knows
                translation_obj = translator.translate(clean_word,src=src,dest=dest)
                translation = translation_obj.text

                # Clean up translation: 
                # remove accents, make lowercase, and remove any non-informative first words
                clean_trans = unidecode(translation)
                clean_trans = clean_trans.lower() 
                g = clean_trans.split(" ")
                if len(g) == 2:
                    if g[0] in ['to','i','we','you','they','he','she','the',"i'll"]:
                        clean_trans = g[1]          
                elif len(g) > 2:
                    if g[1] in ['will']:
                        clean_trans = ' '.join(g[2:len(g)])

                # levenshtein distance features
                levdist = editdistance.eval(clean_word, clean_trans)
                levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                
                # print to console translation and LD 
                print("%s to %s (LD=%i, LDfrac=%.2f)" % (clean_word, clean_trans, levdist, levdistfrac))
                trans_cache[clean_word]=translation
                
                if dest == 'en':
                    aoa_word = clean_trans
                else:
                    aoa_word = clean_word
                word_df = aoa_df[aoa_df['Word'] == aoa_word]
                if not word_df.empty:
                    aoa = word_df['Rating.Mean'].values[0]
                else:
                    aoa = np.nan

                # word frequency of translation (known language) and src words
                trans_freq = word_frequency(translation, dest)
                src_freq = word_frequency(root_token, src)

                # write to csv file
                newrow = [root_token, clean_trans, str(src_freq), str(levdist), str(levdistfrac), str(aoa)]
                newlines.append(",".join(newrow)+'\n')

                
print(len(newlines))
with open(fnout,'w') as fp:
    fp.writelines(newlines)

Will save translations and features to:  es_en_rootwordfeats.txt
el to he (LD=2, LDfrac=1.00)
pan to bread (LD=4, LDfrac=0.80)
agua to water (LD=4, LDfrac=0.80)
ser to be (LD=2, LDfrac=0.67)
mujere to woman (LD=6, LDfrac=1.00)
nosotros to us (LD=7, LDfrac=0.88)
beber to drink (LD=5, LDfrac=1.00)
somo to are (LD=4, LDfrac=1.00)
hombre to man (LD=5, LDfrac=0.83)
los to the (LD=3, LDfrac=1.00)
niño to boy (LD=4, LDfrac=1.00)
ellos to they (LD=5, LDfrac=1.00)
tú to you (LD=3, LDfrac=1.00)
bebe to baby (LD=2, LDfrac=0.50)
venir to come (LD=5, LDfrac=1.00)
yo to i (LD=2, LDfrac=1.00)
bebo to drink (LD=5, LDfrac=1.00)
uno to one (LD=2, LDfrac=0.67)
persona to person (LD=1, LDfrac=0.14)
hola to hello (LD=3, LDfrac=0.60)
gracia to grace (LD=2, LDfrac=0.33)
bueno to good (LD=5, LDfrac=1.00)
día to day (LD=2, LDfrac=0.67)
adiós to goodbye (LD=6, LDfrac=0.86)
perdón to sorry (LD=5, LDfrac=0.83)
por to by (LD=3, LDfrac=1.00)
favor to favor (LD=0, LDfrac=0.00)
sentir to feel (LD=5, LDfrac=0.83)
no t

KeyboardInterrupt: 

In [53]:
# FOR EVERY WORD (NOT root word)
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is

#output filename
fnout = "{}_{}_wordwordfeats.txt".format(src,dest)
print("Will save translations and features to: ",fnout)


trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1 and line[0]!='#':
            # process and translate
            pieces = line.split()
            idx = pieces[0]
            word = pieces[1]
            root_token = pieces[2]
            part_of_speech = pieces[3]
            morph_feats = pieces[4]
            dependency_label = pieces[5]
            dependency_edge_head = pieces[6]

            clean_word = word.lower() # Using WORD not root 
            if '\'' not in clean_word and clean_word not in trans_cache:
                # Wait so as not to annoy the API (otherwise will give JSON error)
                time.sleep(0.2)

                # Translate so we know the word in the language the user already knows
                translation_obj = translator.translate(clean_word,src=src,dest=dest)
                translation = translation_obj.text

                # Clean up translation: 
                # remove accents, make lowercase, and remove any non-informative first words
                clean_trans = unidecode(translation)
                clean_trans = clean_trans.lower() 
                g = clean_trans.split(" ")
                if len(g) == 2:
                    if g[0] in ['to','i','we','you','they','he','she','the']:
                        clean_trans = g[1]          
                elif len(g) > 2:
                    if g[1] in ['will']: # e.g. comere, i will eat
                        clean_trans = ' '.join(g[2:len(g)])

                # levenshtein distance features
                levdist = editdistance.eval(clean_word, clean_trans)
                levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                
                if dest == 'en':
                    aoa_word = clean_trans
                else:
                    aoa_word = clean_word
                word_df = aoa_df[aoa_df['Word'] == aoa_word]
                if not word_df.empty:
                    aoa = word_df['Rating.Mean'].values[0]
                else:
                    aoa = np.nan
                    
                # print to console translation and LD 
                print("%s to %s (LD=%i, LDfrac=%.2f, AOA=%.2f)" % (clean_word, clean_trans, levdist, levdistfrac, aoa))
                trans_cache[clean_word]=translation

                # word frequency of translation (known language) and src words
                trans_freq = word_frequency(translation, dest)
                src_freq = word_frequency(word, src)

                # write to csv file
                newrow = [word, clean_trans, str(src_freq), str(levdist), str(levdistfrac), str(aoa)]
                newlines.append(",".join(newrow)+'\n')

                
print(len(newlines))
with open(fnout,'w') as fp:
    fp.writelines(newlines)

Will save translations and features to:  en_es_wordwordfeats.txt
i to yo (LD=2, LDfrac=1.00, AOA=nan)
am to a.m (LD=1, LDfrac=0.33, AOA=nan)
a to un (LD=2, LDfrac=1.00, AOA=2.89)
boy to chico (LD=5, LDfrac=1.00, AOA=3.67)
from to de (LD=4, LDfrac=1.00, AOA=4.44)
mexico to mejico (LD=1, LDfrac=0.17, AOA=nan)
my to mi (LD=1, LDfrac=0.50, AOA=2.72)
name to nombre (LD=3, LDfrac=0.50, AOA=3.68)
is to es (LD=1, LDfrac=0.50, AOA=nan)
pedro to pedro (LD=0, LDfrac=0.00, AOA=nan)
she to ella (LD=4, LDfrac=1.00, AOA=3.57)
girl to nina (LD=3, LDfrac=0.75, AOA=4.00)
he to el (LD=2, LDfrac=1.00, AOA=3.81)
need to necesitar (LD=6, LDfrac=0.67, AOA=3.56)
taxi to taxi (LD=0, LDfrac=0.00, AOA=7.58)
where to donde (LD=4, LDfrac=0.80, AOA=4.09)
have to tener (LD=4, LDfrac=0.80, AOA=3.72)
reservation to reserva (LD=4, LDfrac=0.36, AOA=11.17)
fine to multa (LD=5, LDfrac=1.00, AOA=7.44)
when to cuando (LD=5, LDfrac=0.83, AOA=4.24)
newspaper to periodico (LD=8, LDfrac=0.89, AOA=5.78)
stop to detener (LD=6, LD

In [50]:
with open(fnout,'w') as fp:
    fp.writelines(newlines)

In [49]:
fnout

'es_en_wordwordfeats.txt'

In [19]:
print("donedone")

donedone


In [14]:
clean_trans = "i will eat"
g = clean_trans.split(" ")
if len(g) == 2:
    if g[0] in ['to','i','we','you','they','he','she','the',"i'll"]:
        clean_trans = g[1]          
elif len(g) > 2:
    if g[1] in ['will']:
        clean_trans = ' '.join(g[2:len(g)])
    print("string with %i parts detected" % (len(g)))
    
print(clean_trans)

string with 3 parts detected
eat


In [21]:
xl = pd.ExcelFile("data/13428_2013_348_MOESM1_ESM.xlsx")
# Print the sheet names
print(xl.sheet_names)

# Load a sheet into a DataFrame by name: df1
df1 = xl.parse('Sheet1')

['Sheet1', 'Sheet2', 'Sheet3']


In [25]:
train_x

NameError: name 'train_x' is not defined

In [24]:
df1['Word']

0                   a
1            aardvark
2              abacus
3             abalone
4             abandon
5           abandoner
6         abandonment
7               abase
8           abasement
9               abate
10          abatement
11           abattoir
12             abbacy
13             abbess
14              abbey
15              abbot
16         abbreviate
17        abbreviated
18       abbreviation
19           abdicate
20         abdication
21            abdomen
22          abdominal
23             abduct
24          abduction
25           abductor
26              abeam
27           aberrant
28         aberration
29               abet
             ...     
31094         zillion
31095       zillionth
31096            zinc
31097       zinfandel
31098            zing
31099          zinger
31100             zip
31101          zipper
31102        zippered
31103           zippy
31104          zircon
31105       zirconium
31106             zit
31107          zither
31108     